# Preparing

In [2]:
# !nvidia-smi

## Install library

In [3]:
# !pip install --quiet pytorch-lightning==1.2.5
# !pip install --quiet tqdm==4.59.0

## Import libraries

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import rc, rcParams
import math

import pandas as pd
import numpy as np
import tqdm
print(tqdm.__version__)
from tqdm.notebook import tqdm
import pytorch_lightning as pl
print(pl.__version__)
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from sklearn.preprocessing import MinMaxScaler

import torch
from torch import autograd, nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from collections import defaultdict

4.61.2
1.2.5


In [5]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALLETE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALLETE))

rcParams['figure.figsize'] = 12, 8

tqdm.pandas()

In [6]:
pl.seed_everything(42)

Global seed set to 42


42

# Data

## Loading watch data

In [7]:
watch = pd.read_csv("Datasets\Price_chart_Tang\Rolex 126334\Combine_all.csv")
watch['Date'] = pd.to_datetime(watch['Date'])
watch['Price'] = pd.to_numeric(watch['Price'])
watch.head()

,Date,Price
0,2017-06-09,9575.0
1,2017-06-10,9575.0
2,2017-06-14,9575.0
3,2017-06-23,9575.0
4,2017-06-24,4791.0


In [8]:
watch = watch.resample('D', on='Date', convention='s').mean()
watch['Date'] = watch.index
watch.reset_index(drop=True, inplace=True)

watch.fillna(method='ffill', inplace=True)

print(watch.shape)
watch.head()

(1736, 2)


,Price,Date
0,9575.0,2017-06-09
1,9575.0,2017-06-10
2,9575.0,2017-06-11
3,9575.0,2017-06-12
4,9575.0,2017-06-13


In [9]:
watch["Prev_Price"] = watch.shift(1)["Price"]
watch.head()
print(type(watch))

<class 'pandas.core.frame.DataFrame'>


In [10]:
watch["Price_Change"] = watch.progress_apply(
    lambda row: 0 if np.isnan(row["Prev_Price"]) else row["Price"] - row["Prev_Price"],
    axis = 1
)
watch.head()

  0%|          | 0/1736 [00:00<?, ?it/s]

,Price,Date,Prev_Price,Price_Change
0,9575.0,2017-06-09,NaN,0.0
1,9575.0,2017-06-10,9575.0,0.0
2,9575.0,2017-06-11,9575.0,0.0
3,9575.0,2017-06-12,9575.0,0.0
4,9575.0,2017-06-13,9575.0,0.0


## Loading other data

In [11]:
gold = pd.read_csv("Datasets\Gold_price.csv")
gold['Gold price (USD)'] = gold['Gold price (USD)'].str.replace(',','')
gold['Date'] = pd.to_datetime(gold['Date'], dayfirst=True)
gold['Gold price (USD)'] = pd.to_numeric(gold['Gold price (USD)'])
mask = (gold.Date >= watch.iloc[0]['Date']) & (gold.Date <= watch.iloc[-1]['Date'])
gold = gold.loc[mask]

gold = gold.resample('D', on='Date', convention='s').mean()
gold.fillna(method='ffill', inplace=True)

print(gold.shape)
gold.head()

(1736, 1)


,Gold price (USD)
Date,
2017-06-09,1266.6
2017-06-10,1266.6
2017-06-11,1266.6
2017-06-12,1266.4
2017-06-13,1262.0


## Create feature df

In [12]:
rows = []

for _, row in tqdm(watch.iterrows(), total=watch.shape[0]):
    row_data = dict(
        day_of_week = row.Date.dayofweek,
        day_of_month = row.Date.day,
        week_of_year = row.Date.week,
        month = row.Date.month,
        price_change = row.Price_Change,
        price = row.Price
    )
    rows.append(row_data)

features_df = pd.DataFrame(rows)

  0%|          | 0/1736 [00:00<?, ?it/s]

In [13]:
print(features_df.shape)
features_df.head()

(1736, 6)


,day_of_week,day_of_month,week_of_year,month,price_change,price
0,4,9,23,6,0.0,9575.0
1,5,10,23,6,0.0,9575.0
2,6,11,23,6,0.0,9575.0
3,0,12,24,6,0.0,9575.0
4,1,13,24,6,0.0,9575.0


## Set TRAIN_RATIO

In [14]:
TRAIN_RATIO = 0.6

train_size = int(len(features_df) * TRAIN_RATIO)
print(train_size)

1041


In [15]:
train_df, test_df = features_df[:train_size], features_df[train_size:]
print(train_df.shape, test_df.shape)

(1041, 6) (695, 6)


In [16]:
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler = scaler.fit(train_df)

In [17]:
train_df = pd.DataFrame(
    scaler.transform(train_df),
    index=train_df.index,
    columns=train_df.columns
)

In [18]:
test_df = pd.DataFrame(
    scaler.transform(test_df),
    index=test_df.index,
    columns=test_df.columns
)

In [19]:
test_df.head()

,day_of_week,day_of_month,week_of_year,month,price_change,price
1041,-0.333333,-0.066667,-0.411765,-0.454545,0.395268,0.926580
1042,0.000000,0.000000,-0.411765,-0.454545,0.407171,0.951315
1043,0.333333,0.066667,-0.411765,-0.454545,0.388881,0.951315
1044,0.666667,0.133333,-0.411765,-0.454545,0.388881,0.951315
1045,1.000000,0.200000,-0.411765,-0.454545,0.388881,0.951315


In [20]:
def create_sequences(input_data: pd.DataFrame, target_column, sequence_length, days_pred):

    sequences = []
    data_size = len(input_data)

    for i in tqdm(range(data_size - sequence_length - days_pred)):

        sequence = input_data[i:i+sequence_length]

        label_position = i + sequence_length + days_pred
        label = input_data.iloc[label_position][target_column]
        
        sequences.append((sequence, label))

    return sequences

## Set SEQUENCE_LENGTH and Created sequences

In [21]:
SEQUENCE_LENGTH = 30
DAYS_PREDICTION = 30

train_sequences = create_sequences(train_df, "price", SEQUENCE_LENGTH, DAYS_PREDICTION)
test_sequences = create_sequences(test_df, "price", SEQUENCE_LENGTH, DAYS_PREDICTION)

  0%|          | 0/981 [00:00<?, ?it/s]

  0%|          | 0/635 [00:00<?, ?it/s]

In [22]:
print(f"Sequence shape: {train_sequences[0][0].shape}")
print(f"Train length: {len(train_sequences)}")
print(f"Test length: {len(test_sequences)}")
print(f"Test length: {train_sequences[0][1]}")
train_sequences[0][0].head()

Sequence shape: (30, 6)
Train length: 981
Test length: 635
Test length: -0.17353749509226546


,day_of_week,day_of_month,week_of_year,month,price_change,price
0,0.333333,-0.466667,-0.137255,-0.090909,0.388881,0.878288
1,0.666667,-0.400000,-0.137255,-0.090909,0.388881,0.878288
2,1.000000,-0.333333,-0.137255,-0.090909,0.388881,0.878288
3,-1.000000,-0.266667,-0.098039,-0.090909,0.388881,0.878288
4,-0.666667,-0.200000,-0.098039,-0.090909,0.388881,0.878288


# Classes create

## Set device

In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = 'cpu'
print("configured device:", device)

configured device: cuda


## TS_Dataset (Time series)

In [24]:
class TS_Dataset(Dataset):
    def __init__(self, sequences):
        self.sequences = sequences

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        
        sequence, label = self.sequences[idx]

        return dict(
            sequence = torch.Tensor(sequence.to_numpy()).to(device),
            label = torch.tensor(label).float().to(device)
        )

## Price Data Module

In [25]:
class PriceDataModule(pl.LightningDataModule):
    def __init__(self, train_sequences, test_sequences, batch_size=8):
        super().__init__
        self.train_sequences = train_sequences
        self.test_sequences = test_sequences
        self.batch_size = batch_size
        
    def prepare_data(self):
        self._has_prepared_data = True

    def setup(self):
        self.train_dataset = TS_Dataset(self.train_sequences)
        self.test_dataset = TS_Dataset(self.test_sequences)
        self.prepare_data()
        
    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=False
        )
    
    def val_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=4,
            shuffle=False
        )
        
    def test_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=1,
            shuffle=False
        )

## Define BATCH_SIZE and N_EPOCHS

In [26]:
N_EPOCHS = 8
BATCH_SIZE = 128

data_module = PriceDataModule(train_sequences, test_sequences, batch_size=BATCH_SIZE)
data_module.setup()

In [27]:
train_dataset = TS_Dataset(train_sequences)

In [28]:
for item in train_dataset:
    print(item["sequence"].shape)
    print(item["label"].shape)
    print(item["label"])
    break

torch.Size([30, 6])
torch.Size([])
tensor(-0.1735, device='cuda:0')


## Price Prediction Model

In [29]:
class PricePredictionModel(nn.Module):

    def __init__(self, n_features, n_hidden=128, n_layers=2):
        super().__init__()
        self.n_hidden = n_hidden
        self.lstm = nn.LSTM(
            input_size = n_features,
            hidden_size =n_hidden,
            batch_first = True,
            num_layers = n_layers,
            dropout = 0.2
        )
        self.regressor = nn.Linear(n_hidden, 1)

    def forward(self, x):
        self.lstm.flatten_parameters()

        _, (hidden, _) = self.lstm(x)
        out = hidden[-1]

        return self.regressor(out)

## Price Predictor

In [30]:
class PricePredictor(pl.LightningModule):

    def __init__(self, n_features: int):
        super().__init__()
        self.model = PricePredictionModel(n_features).to(device)
        self.criterion = nn.MSELoss().to(device)

    def forward(self, x, labels=None):
        output = self.model(x)
        loss = 0
        if labels is not None:
            loss = self.criterion(output, labels.unsqueeze(dim=1))

        return loss, output
    
    def training_step(self, batch, batch_idx):
        sequences = batch["sequence"]
        labels = batch["label"]
        loss, outputs = self(sequences, labels)
        self.log("train_loss", loss, prog_bar=True, logger=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        sequences = batch["sequence"]
        labels = batch["label"]
        loss, outputs = self(sequences, labels)
        self.log("val_loss", loss, prog_bar=True, logger=True)
        return loss    

    def test_step(self, batch, batch_idx):
        sequences = batch["sequence"]
        labels = batch["label"]
        loss, outputs = self(sequences, labels)
        self.log("test_loss", loss, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        return optim.Adagrad(self.parameters(), lr=1e-3)

In [31]:
model = PricePredictor(n_features=train_df.shape[1])

In [32]:
for item in data_module.train_dataloader():
    print(item["sequence"].shape)
    print(item["label"].shape)
    break

torch.Size([128, 30, 6])
torch.Size([128])


# Start Tensorboard

In [33]:
%load_ext tensorboard
%tensorboard --logdir ./lightning_logs

Reusing TensorBoard on port 6006 (pid 17812), started 4 days, 12:18:59 ago. (Use '!kill 17812' to kill it.)

## checkpoint_callback setup

In [34]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    filename="best-checkpoint",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min"
)

logger = TensorBoardLogger("lightning_logs", name="watch-price")

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=2)

trainer = pl.Trainer(
    logger=logger,
    checkpoint_callback=checkpoint_callback,
    callbacks=[early_stopping_callback],
    max_epochs=N_EPOCHS,
    gpus=1,
    progress_bar_refresh_rate=30
)

C:\Users\THETANG\anaconda3\envs\BIA_env\lib\site-packages\pytorch_lightning\utilities\distributed.py:52: UserWarning: Checkpoint directory checkpoints exists and is not empty.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: None, using: 0 TPU cores


## Fit

In [35]:
# trainer.fit(model, data_module)

# Evaluation

## Load trained model

In [36]:
trained_model = PricePredictor.load_from_checkpoint(
    "checkpoints/best-checkpoint.ckpt",
    n_features=train_df.shape[1]
)

In [37]:
trained_model.freeze()

## Predictions

In [38]:
test_dataset = TS_Dataset(test_sequences)

predictions = []
labels = []

for item in tqdm(test_dataset):
    sequence = item['sequence']
    label = item['label']

    _, output = trained_model(sequence.unsqueeze(dim=0))
    predictions.append(output.item())
    labels.append(label.item())

  0%|          | 0/635 [00:00<?, ?it/s]

In [39]:
len(predictions), len(test_df)-SEQUENCE_LENGTH

(635, 665)

## Descale

In [40]:
def descale(descaler, values):
    values_2d = np.array(values)[:, np.newaxis]
    return descaler.inverse_transform(values_2d).flatten()

In [41]:
descaler = MinMaxScaler()
descaler.min_, descaler.scale_ = scaler.min_[-1], scaler.scale_[-1]

predictions_descaled = descale(descaler, predictions)
labels_descaled = descale(descaler, labels)

print("Predictions: ", predictions_descaled[:5])
print("Labels: ", labels_descaled[:5])

Predictions:  [9454.29808348 9452.24314231 9454.50424558 9459.78141832 9453.97957975]
Labels:  [9590.99994832 9630.99994916 9630.99994916 9630.99994916 9617.00005513]


## Plot graph

In [42]:
# train_data = watch[:train_size]
test_data = watch[train_size:]
print(test_data.shape)
print(test_df.shape)

(695, 4)
(695, 6)


In [43]:
test_sequences_data = test_data.iloc[SEQUENCE_LENGTH:]
print(test_sequences_data.shape)

(665, 4)


In [51]:
test_dates = matplotlib.dates.date2num(test_sequences_data.Date.tolist())
all_dates = matplotlib.dates.date2num(watch.Date.tolist())
plt.plot_date(test_dates[DAYS_PREDICTION:], predictions_descaled, '-', label='predicted')
# plt.plot_date(test_dates, labels_descaled, '-', label='real')
plt.plot_date(all_dates, watch.Price.tolist(), '-', label='real')
i = 150
plt.plot_date([test_dates[i], test_dates[DAYS_PREDICTION+i]], [labels_descaled[i-DAYS_PREDICTION], predictions_descaled[i]], '-', label='DAYS_PREDICTION')

plt.xticks(rotation=45)
plt.legend()
plt.show()

# Evalution function

In [ ]:
# class prediction:
#     def __init__(self, model_path, n_features):
#         self.trained_model = PricePredictor.load_from_checkpoint(
#             model_path,
#             n_features=n_features
#         )
#         self.trained_model.freeze()

#     def get_result(self, sequence, days):
#         _, predictions = trained_model(sequence.unsqueeze(dim=0))
        
